# Advent of Code 2017
Solutions to [Advent of Code 2017](http://adventofcode.com/2017).

These aren't as I first wrote them. I renamed functions and variables, added comments, and removed debugging print statements – some that I'd already commented out and some not.

I also extracted the following functions from various days, so that I could re-use them for subsequent days.

In [1]:
def example(actual, expected=True):
    if actual != expected:
        print('Actual = {} ≠ {} = expected'.format(actual, expected))

def examples(fn, *pairs):
    assert len(pairs) % 2 == 0
    for i in range(0, len(pairs), 2):
        arg, expected = pairs[i:i+2]
        example(fn(arg), expected)
        
def iter_len(iterable):
    return sum(1 for _ in iterable)

Helpers for here data and input files:

In [2]:
def here_lines(s):
    return s.lstrip().splitlines()

def input_text(day):
    "Return the data for the given day."
    return open('data/advent-2017/input-{}.txt'.format(day)).read().strip()

def input_lines(day):
    return input_text(day).splitlines()

def count_filtered_inputs(pred, day):
    return sum(map(pred, input_lines(day)))

## [Day 1: Inverse Captcha](http://adventofcode.com/2017/day/1)

In [3]:
def sum_same_as_next(digits):
    return sum(int(c)
               for i, c in enumerate(digits)
               if c == digits[(i + 1) % len(digits)])

examples(sum_same_as_next,
         '1122', 3,
         '1111', 4,
         '1234', 0,
         '91212129', 9)

sum_same_as_next(input_text(1))

1119

### Part 2
If I'd seen Part 2 before I wrote Part 1, I would have defined `sum_same_as_next` in terms of `sum_same_as_antipode`.

In [4]:
def sum_same_as_antipode(digits, delta=None):
    delta = delta or len(digits) // 2
    return sum(int(c)
               for i, c in enumerate(digits)
               if c == digits[(i + delta) % len(digits)])

examples(sum_same_as_antipode,
         '1212', 6,
         '1221', 0,
         '123425', 4,
         '123123', 12,
         '12131415', 4)

sum_same_as_antipode(input_text(1))

1420

### Alternatives
Here's some alternate implementations that I also had in my head when I wrote the one above. With less time pressure, I'd write them all, and keep the one that looked more maintainable.

These can trivially be extended to `sum_same_as_antipode`.

In [5]:
def sum_same_as_next(digits):
    buffered = digits + digits[:1]
    return sum(int(c)
               for i, c in enumerate(digits)
               if c == buffered[i + 1])

def sum_same_as_next(digits):
    return sum(int(c1)
               for c1, c2 in zip(digits, digits[1:] + digits[:1])
               if c1 == c2)

examples(sum_same_as_next,
         '1122', 3,
         '1111', 4,
         '1234', 0,
         '91212129', 9)

sum_same_as_next(input_text(1))

1119

## [Day 2: Corruption Checksum](http://adventofcode.com/2017/day/2)

In [6]:
def spreadsheet(text):
    "Given a text matrix with newlines, return a list of list of ints."
    return [list(map(int, line.split()))
            for line in text.splitlines()
            if line]

def checksum(sheet):
    return sum(max(row) - min(row) for row in spreadsheet(sheet))

example(checksum('''
5 1 9 5
7 5 3
2 4 6 8'''), 18)

checksum(input_text(2))

41887

### Part 2

In [7]:
from itertools import combinations

def evenly(gridstr):
    ints = lambda ns: next(p // q for q, p in combinations(sorted(set(ns)), 2) if p % q == 0)
    return sum(map(ints, spreadsheet(sheet)))

example(evenly('''
5 9 2 8
9 4 7 3
3 8 6 5'''), 9)

evenly(input_text(2))

226

## [Day 3: Spiral Memory](http://adventofcode.com/2017/day/3)

In [8]:
from itertools import accumulate, chain, count, repeat

def spiral_steps():
    d = 1
    for w in count(1):
        yield from repeat(d, w)
        d *= 1j
        yield from repeat(d, w)
        d *= 1j

def spiral_positions():
    "Generate (x, y) position along the spiral, starting at (0, 0)."
    def c_to_pair(c):
        return (int(c.real), int(c.imag))
    return map(c_to_pair, accumulate(chain([0j], spiral_steps())))

def nth_spiral_position(n):
    "Return the x, y position of the cell that holds n."
    return next(pos for i, pos in enumerate(spiral_positions(), 1) if i == n)

def steps(n):
    "Return the Manhattan distance from 1-based nth element to the center."
    x, y = nth_spiral_position(n)
    return abs(x) + abs(y)

def print_spiral(w):
    cells = {}
    for n, pos in enumerate(spiral_positions(), 1):
        x, y = pos
        if max(abs(x), abs(y)) > w:
            break
        cells[pos] = n
    for y in range(w, -w-1, -1):
        ns = [cells.get((x, y), None) for x in range(-w, w+1)]
        xs = ['{:4d}'.format(n) if n else '    ' for n in ns]
        print(' '.join(xs))

print_spiral(3)

examples(steps,
         1, 0,
         12, 3,
         23, 2,
         1024, 31)

steps(361527)

  37   36   35   34   33   32   31
  38   17   16   15   14   13   30
  39   18    5    4    3   12   29
  40   19    6    1    2   11   28
  41   20    7    8    9   10   27
  42   21   22   23   24   25   26
  43   44   45   46   47   48   49


326

### Part 2

In [9]:
from collections import defaultdict

def neighbors(pos):
    x, y = pos
    return ((x + dx, y + dy)
            for dx in (-1, 0, 1)
            for dy in (-1, 0, 1)
            if dx or dy)

def spiral_sums():
    m = defaultdict(int)
    for pos in spiral_positions():
        a = sum(m[n] for n in neighbors(pos)) or 1
        m[pos] = a
        yield a

next(n for n in spiral_sums() if n > 361527)

363010

### Alternatives

I kind of like these better, but they read more like Haskell, less like idiomatic Python (as evident from the imports).

Where I to start over, I'd change `spiral_positions` to return complex numbers instead of tuples, and change its consumers to expect them.

In [10]:
from functools import reduce
from itertools import accumulate, chain, islice
from operator import add

def spiral_positions():
    turns = (1j ** n for n in count())
    sides = (w for n in count(1) for w in (n, n))  # number of steps in each direction
    steps = (d for n, d in zip(sides, turns) for _ in range(n))
    yield (0, 0)
    yield from ((int(c.real), int(c.imag)) for c in accumulate(steps, add))

def print_spiral(w):
    h = w // 2
    cells = {pos: n for n, pos in enumerate(islice(spiral_positions(), w**2), 1)}
    row = lambda y:[cells.get((x, y), '') for x in range(-h, h+1)]
    cell_width = max(len(str(n)) for n in cells.values())
    def fmt(n): return '{:{}}'.format(n, cell_width)
    for y in range(h, -h-1, -1):
        print(' '.join(map(fmt, row(y))))

print_spiral(7)

37 36 35 34 33 32 31
38 17 16 15 14 13 30
39 18  5  4  3 12 29
40 19  6  1  2 11 28
41 20  7  8  9 10 27
42 21 22 23 24 25 26
43 44 45 46 47 48 49


## [Day 4: High-Entropy Passphrases](http://adventofcode.com/2017/day/4)

In [11]:
def valid_passphrase(phrase):
    ws = phrase.split()
    return len(ws) == len(set(ws))

examples(valid_passphrase,
         'aa bb cc dd ee', True,
         'aa bb cc dd aa', False,
         'aa bb cc dd aaa', True)

count_filtered_inputs(valid_passphrase, 4)

383

In [12]:
def valid_passphrase(phrase):
    ws = phrase.split()
    return len(ws) == len(set(''.join(sorted(w)) for w in ws))

examples(valid_passphrase,
         'abcde fghij', True,
         'abcde xyz ecdab', False,
         'a ab abc abd abf abj', True,
         'iiii oiii ooii oooi oooo', True,
         'oiii ioii iioi iiio', False)

count_filtered_inputs(valid_passphrase, 4)

265

## [Day 5: A Maze of Twisty Trampolines, All Alike](http://adventofcode.com/2017/day/5)

In [13]:
def iter_jumps(lines):
    jumps = list(map(int, lines))
    pc = 0
    while 0 <= pc < len(jumps):
        yield pc
        n = jumps[pc]
        jumps[pc] += 1
        pc += n

jumps = here_lines('''
0
3
0
1
-3''')

def jump_count(lines):
    return iter_len(iter_jumps(lines))

example(jump_count(jumps), 5)
jump_count(input_lines(5))

372139

In [14]:
def iter_jumps(lines):
    jumps = list(map(int, lines))
    pc = 0
    while 0 <= pc < len(jumps):
        yield pc
        n = jumps[pc]
        jumps[pc] += -1 if n >= 3 else 1
        pc += n

example(jump_count(jumps), 10)
jump_count(input_lines(5))

29629538